In [2]:
import pandas as pd

# 读取电影数据
movies_df = pd.read_csv('movies.csv')
# 读取人物数据
person_df = pd.read_csv('person.csv')
# 读取电影与人物关系数据
relationships_df = pd.read_csv('relationships.csv')

In [3]:
# 根据角色类型筛选导演、编剧和演员
directors_df = relationships_df[relationships_df['role'] == 'director'][['movie_id', 'person_id']]
authors_df = relationships_df[relationships_df['role'] == 'author'][['movie_id', 'person_id']]
actors_df = relationships_df[relationships_df['role'] == 'actor'][['movie_id', 'person_id']]

In [4]:
directors_df = pd.merge(directors_df, person_df, on='person_id', how='left')[['movie_id', 'name']]
directors_df = directors_df.rename(columns={'name': 'director_name'})
authors_df = pd.merge(authors_df, person_df, on='person_id', how='left')[['movie_id', 'name']]
authors_df = authors_df.rename(columns={'name': 'author_name'})
actors_df = pd.merge(actors_df, person_df, on='person_id', how='left')[['movie_id', 'name']]
actors_df = actors_df.rename(columns={'name': 'actor_name'})

In [5]:
directors_df['director_name'] = directors_df['director_name'].apply(lambda x: [x] if isinstance(x, str) else x)
directors_df = directors_df.groupby('movie_id')['director_name'].apply(lambda x: [item for sublist in x for item in sublist]).reset_index()
directors_df.reset_index(drop=True, inplace=True)
# 现在，directors_df 中的每个 movie_id 应该只有一个对应的 director_name 列表
authors_df['author_name'] = authors_df['author_name'].apply(lambda x: [x] if isinstance(x, str) else x)
authors_df = authors_df.groupby('movie_id')['author_name'].apply(lambda x: [item for sublist in x for item in sublist]).reset_index()
authors_df.reset_index(drop=True, inplace=True)
# authors_df 中的每个 movie_id 应该只有一个对应的 author_name 列表
actors_df['actor_name'] = actors_df['actor_name'].apply(lambda x: [x] if pd.notna(x) and isinstance(x, str) else [])
actors_df['actor_name'] = actors_df['actor_name'].apply(lambda x: [x] if isinstance(x, str) else x)
actors_df = actors_df.groupby('movie_id')['actor_name'].apply(lambda x: [item for sublist in x for item in sublist]).reset_index()
actors_df.reset_index(drop=True, inplace=True)
# actors_df 中的每个 movie_id 应该只有一个对应的 actor_name 列表

In [ ]:
print(directors_df.head)
print(authors_df.head)
print(actors_df.head)

In [7]:
# 将筛选后的数据合并到电影数据中
movies_df = pd.merge(movies_df, directors_df, on='movie_id', how='left', suffixes=('', '_director'))
movies_df = pd.merge(movies_df, authors_df, on='movie_id', how='left', suffixes=('', '_author'))
movies_df = pd.merge(movies_df, actors_df, on='movie_id', how='left', suffixes=('', '_actor'))

In [11]:
# 保存更新后的电影数据到新的CSV文件
movies_df.to_csv('merged.csv', index=False)
# 获取前100行数据
top_100_movies_df = movies_df.head(100)
# 保存到 CSV 文件，不包括索引
top_100_movies_df.to_csv('merged_partial.csv', index=False)